# 12.B.1: ASSIGNMENT

PROBLEM STATEMENT



Write REST APIs using DRF for managing music records.
We should be able to add/read/update/delete: (read, update and delete using id)

An artist (id, name, dob, genre)

A genre (id, name)

A track (id, name, artist, duration(in minutes), year of release)


We should also be able to get:

A list of all tracks from an artist

A list of all artists

A list of all tracks

### First, you'll need to install Django and DRF. You can do this by running the following commands in your terminal:

In [ ]:
pip install django
pip install djangorestframework


### Once you have installed these libraries, you can create a new Django project by running the following command:

In [ ]:
django-admin startproject music_records


### Then, create a new app called music by running the following command:

In [ ]:
cd music_records
python manage.py startapp music


### Now, let's define our models in the models.py file of the music app:

In [ ]:
from django.db import models

class Genre(models.Model):
    name = models.CharField(max_length=50)

class Artist(models.Model):
    name = models.CharField(max_length=50)
    dob = models.DateField()
    genre = models.ForeignKey(Genre, on_delete=models.CASCADE)

class Track(models.Model):
    name = models.CharField(max_length=50)
    artist = models.ForeignKey(Artist, on_delete=models.CASCADE)
    duration = models.IntegerField()
    year = models.IntegerField()


### Next, we need to create serializers to convert our models to JSON format. Create a new file called serializers.py in the music app:

In [ ]:
from rest_framework import serializers
from .models import Genre, Artist, Track

class GenreSerializer(serializers.ModelSerializer):
    class Meta:
        model = Genre
        fields = '__all__'

class ArtistSerializer(serializers.ModelSerializer):
    class Meta:
        model = Artist
        fields = '__all__'

class TrackSerializer(serializers.ModelSerializer):
    class Meta:
        model = Track
        fields = '__all__'


### Now, let's create the views to handle our API requests. Create a new file called views.py in the music app:

In [ ]:
from rest_framework import generics
from .models import Genre, Artist, Track
from .serializers import GenreSerializer, ArtistSerializer, TrackSerializer

class GenreListCreateView(generics.ListCreateAPIView):
    queryset = Genre.objects.all()
    serializer_class = GenreSerializer

class GenreRetrieveUpdateDestroyView(generics.RetrieveUpdateDestroyAPIView):
    queryset = Genre.objects.all()
    serializer_class = GenreSerializer

class ArtistListCreateView(generics.ListCreateAPIView):
    queryset = Artist.objects.all()
    serializer_class = ArtistSerializer

class ArtistRetrieveUpdateDestroyView(generics.RetrieveUpdateDestroyAPIView):
    queryset = Artist.objects.all()
    serializer_class = ArtistSerializer

class TrackListCreateView(generics.ListCreateAPIView):
    queryset = Track.objects.all()
    serializer_class = TrackSerializer

class TrackRetrieveUpdateDestroyView(generics.RetrieveUpdateDestroyAPIView):
    queryset = Track.objects.all()
    serializer_class = TrackSerializer

class TrackListByArtistView(generics.ListAPIView):
    serializer_class = TrackSerializer

    def get_queryset(self):
        artist_id = self.kwargs['artist_id']
        return Track.objects.filter(artist__id=artist_id)

class ArtistListView(generics.ListAPIView):
    queryset = Artist.objects.all()
    serializer_class = ArtistSerializer

class TrackListView(generics.ListAPIView):
    queryset = Track.objects.all()
    serializer_class = TrackSerializer


### Finally, let's define the URLs for our API. Create a new file called urls.py in the music app:

In [ ]:
from django.urls import path
from .views import GenreListCreateView, GenreRetrieveUpdateDestroyView, \
    ArtistListCreateView, ArtistRetrieveUpdateDestroyView, \
    TrackListCreateView, TrackRetrieve


# 12.B.2: ASSIGNMENT

PROBLEM STATEMENT



Create a student record management system with APIs for:


- Adding a new student (with name, address, grade)

- Adding test records for current grade in 3 subjects science, maths and english

- Returning a list of top n rankers of a grade where n and grade number are passed in request body JSON

- Returning a list of top 3 rankers form all grades

Structuring DB and the number of grades is up to student (with a minimum of 3 grades)

### First, you'll need to install Django and DRF. You can do this by running the following commands in your terminal:

In [ ]:
pip install django
pip install djangorestframework


### Once you have installed these libraries, you can create a new Django project by running the following command:

In [ ]:
django-admin startproject student_records


### Then, create a new app called students by running the following command:

In [ ]:
cd student_records
python manage.py startapp students


### Now, let's define our models in the models.py file of the students app:

In [ ]:
from django.db import models

class Grade(models.Model):
    grade_number = models.IntegerField()

class Student(models.Model):
    name = models.CharField(max_length=50)
    address = models.CharField(max_length=200)
    grade = models.ForeignKey(Grade, on_delete=models.CASCADE)

class TestRecord(models.Model):
    subject = models.CharField(max_length=50)
    score = models.FloatField()
    student = models.ForeignKey(Student, on_delete=models.CASCADE)


### Next, let's create serializers for our models in serializers.py:

In [ ]:
from rest_framework import serializers
from .models import Grade, Student, TestRecord

class GradeSerializer(serializers.ModelSerializer):
    class Meta:
        model = Grade
        fields = '__all__'

class TestRecordSerializer(serializers.ModelSerializer):
    class Meta:
        model = TestRecord
        fields = '__all__'

class StudentSerializer(serializers.ModelSerializer):
    grade = GradeSerializer()
    test_records = TestRecordSerializer(many=True)

    class Meta:
        model = Student
        fields = '__all__'


### Now, let's define our views in views.py:

In [ ]:
from rest_framework import generics, status
from rest_framework.response import Response
from .models import Grade, Student, TestRecord
from .serializers import GradeSerializer, StudentSerializer, TestRecordSerializer

class CreateGradeView(generics.CreateAPIView):
    serializer_class = GradeSerializer

class CreateStudentView(generics.CreateAPIView):
    serializer_class = StudentSerializer

class CreateTestRecordView(generics.CreateAPIView):
    serializer_class = TestRecordSerializer

class GetRankersByGradeView(generics.ListAPIView):
    serializer_class = StudentSerializer

    def get_queryset(self):
        grade_number = self.request.data.get('grade_number')
        n = self.request.data.get('n')
        queryset = Student.objects.filter(grade__grade_number=grade_number).annotate(avg_score=(models.Sum('testrecord__score')/models.Count('testrecord__score'))).order_by('-avg_score')[:n]
        return queryset

class GetTop3RankersView(generics.ListAPIView):
    serializer_class = StudentSerializer

    def get_queryset(self):
        queryset = Student.objects.annotate(avg_score=(models.Sum('testrecord__score')/models.Count('testrecord__score'))).order_by('-avg_score')[:3]
        return queryset


### Finally, let's define our urls in urls.py:

In [ ]:
from django.urls import path
from .views import CreateGradeView, CreateStudentView, CreateTestRecordView, GetRankersByGradeView, GetTop3RankersView

urlpatterns = [
    path('grades/', CreateGradeView.as_view(), name='create-grade'),
    path('students/', CreateStudentView.as_view(), name='create-student'),
    path('test-records/', CreateTestRecordView.as_view(), name='create-test-record'),
    path('rankers-by-grade/', GetRankersByGradeView.as_view(), name='rankers-by-grade'),
    path('top-3-rankers/', GetTop3RankersView.as_view(), name='top-3-rankers'),
]
